In [ ]:
import torch
from torch import nn
from torchvision import models, transforms
from PIL import Image
import os

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load class names (make sure these match your training setup)
class_names = ["cats", "dogs"]  # Replace with actual class names

In [ ]:
# Load the trained model
model = models.resnet18(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 2),  # Binary classification
)

In [ ]:
# Load the trained model
model.load_state_dict(torch.load("cat_dog_resnet18.pth", map_location=device))
model = model.to(device)  # Move the model to the desired device
model.eval()

In [ ]:
# Define the image transformation pipeline
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

In [ ]:
# Directory containing test images
test_image_dir = "test_images"  # Replace with the path to your test images
test_images = [
    os.path.join(test_image_dir, img)
    for img in os.listdir(test_image_dir)
    if img.endswith(("png", "jpg", "jpeg"))
]

test_images

In [ ]:
# Perform inference on each image
for img_path in test_images:
    # Load and transform the image
    image = Image.open(img_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)

    # Get class name
    class_name = class_names[predicted.item()]

    print(
        f"Image: {img_path} --> Predicted Class: {class_name}, Confidence: {confidence.item() * 100:.2f}%"
    )